In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
bracket_df = pd.read_csv('Data_cleaning/barttorvik_all_years.csv')
bracket_df.head()

,Year,Seed,School,Round Finished,Conf,G,Rec,AdjOE,AdjDE,AdjNR,...,FTR,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB
0,2008,1,Kansas,6.0,B12,33,30–3,121.1,85.9,35.2,...,37.5,30.8,54.8,40.9,39.9,34.0,29.2,38.1,69.5,9.9
1,2008,1,Memphis,5.0,CUSA,34,33–1,113.4,84.0,29.4,...,38.6,32.7,53.3,41.6,35.0,29.8,36.7,28.7,70.7,9.0
2,2008,1,UCLA,4.0,P10,33,30–3,116.4,87.2,29.2,...,38.0,25.7,52.9,46.8,33.8,34.2,28.1,27.9,66.2,10.7
3,2008,3,Wisconsin,2.0,B10,33,29–4,112.3,85.2,27.1,...,39.3,25.4,49.2,41.3,35.9,31.1,31.1,32.8,63.5,8.2
4,2008,2,Duke,1.0,ACC,32,27–5,117.3,89.0,28.3,...,40.4,32.0,51.8,47.1,38.4,32.8,39.1,24.8,73.7,8.9


In [2]:
#Exclude 2025 data from CSV

#Create Copy of Data
copy_df = bracket_df.copy()

#Create New Dataframe for all years except 2025
no_2025_df = copy_df.loc[(copy_df["Year"] != 2025)]


In [3]:
#Use list comprehension to create Elite 8 values

e_8 = [1 if x >= 3 else 0 for x in no_2025_df['Round Finished']]

#Assign list to column
no_2025_df['Elite 8'] = e_8
#Show Dataframe
no_2025_df.head()

/var/folders/f6/crwn6xns7ng5zdqghg60td240000gn/T/ipykernel_11190/2693226293.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_2025_df['Elite 8'] = e_8


,Year,Seed,School,Round Finished,Conf,G,Rec,AdjOE,AdjDE,AdjNR,...,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Elite 8
0,2008,1,Kansas,6.0,B12,33,30–3,121.1,85.9,35.2,...,30.8,54.8,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1
1,2008,1,Memphis,5.0,CUSA,34,33–1,113.4,84.0,29.4,...,32.7,53.3,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1
2,2008,1,UCLA,4.0,P10,33,30–3,116.4,87.2,29.2,...,25.7,52.9,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1
3,2008,3,Wisconsin,2.0,B10,33,29–4,112.3,85.2,27.1,...,25.4,49.2,41.3,35.9,31.1,31.1,32.8,63.5,8.2,0
4,2008,2,Duke,1.0,ACC,32,27–5,117.3,89.0,28.3,...,32.0,51.8,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0


In [4]:
#Drop Round Finished from DataFrame
e_8_df = no_2025_df.drop(columns='Round Finished')

e_8_df.head()

,Year,Seed,School,Conf,G,Rec,AdjOE,AdjDE,AdjNR,Barthag,...,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Elite 8
0,2008,1,Kansas,B12,33,30–3,121.1,85.9,35.2,0.9810,...,30.8,54.8,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1
1,2008,1,Memphis,CUSA,34,33–1,113.4,84.0,29.4,0.9692,...,32.7,53.3,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1
2,2008,1,UCLA,P10,33,30–3,116.4,87.2,29.2,0.9653,...,25.7,52.9,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1
3,2008,3,Wisconsin,B10,33,29–4,112.3,85.2,27.1,0.9599,...,25.4,49.2,41.3,35.9,31.1,31.1,32.8,63.5,8.2,0
4,2008,2,Duke,ACC,32,27–5,117.3,89.0,28.3,0.9597,...,32.0,51.8,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0


In [5]:
#Split Record into Wins and Losses
e_8_df[['Wins', 'Losses']] = e_8_df['Rec'].str.split('–', expand=True)

#Change Columns to Ints
e_8_df['Wins'] = e_8_df['Wins'].astype(int)
e_8_df['Losses'] = e_8_df['Losses'].astype(int)

#Drop Rec Column
final_df = e_8_df.drop(columns=['Rec', 'School', 'Conf'])

final_df.head()

,Year,Seed,G,AdjOE,AdjDE,AdjNR,Barthag,EFG%,EFGD%,TOR,...,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Elite 8,Wins,Losses
0,2008,1,33,121.1,85.9,35.2,0.9810,56.3,44.8,18.7,...,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1,30,3
1,2008,1,34,113.4,84.0,29.4,0.9692,53.0,42.5,17.2,...,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1,33,1
2,2008,1,33,116.4,87.2,29.2,0.9653,52.3,48.0,18.8,...,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1,30,3
3,2008,3,33,112.3,85.2,27.1,0.9599,50.6,43.1,19.4,...,41.3,35.9,31.1,31.1,32.8,63.5,8.2,0,29,4
4,2008,2,32,117.3,89.0,28.3,0.9597,54.1,47.6,18.2,...,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0,27,5


In [6]:
#Create Dummies Dataframe
dummies_df = pd.get_dummies(final_df)
dummies_df.head()

,Year,Seed,G,AdjOE,AdjDE,AdjNR,Barthag,EFG%,EFGD%,TOR,...,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Elite 8,Wins,Losses
0,2008,1,33,121.1,85.9,35.2,0.9810,56.3,44.8,18.7,...,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1,30,3
1,2008,1,34,113.4,84.0,29.4,0.9692,53.0,42.5,17.2,...,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1,33,1
2,2008,1,33,116.4,87.2,29.2,0.9653,52.3,48.0,18.8,...,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1,30,3
3,2008,3,33,112.3,85.2,27.1,0.9599,50.6,43.1,19.4,...,41.3,35.9,31.1,31.1,32.8,63.5,8.2,0,29,4
4,2008,2,32,117.3,89.0,28.3,0.9597,54.1,47.6,18.2,...,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0,27,5


In [7]:
#separate Data into features and target arrays
y = dummies_df["Elite 8"]
X = dummies_df.drop(columns="Elite 8")

#Split the data into training and testing dataset\
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=25))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 15)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [10]:
#Import Keras Tuner
import keras_tuner as kt

#Create Tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [11]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

In [12]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 11, 'num_layers': 2, 'units_0': 16, 'units_1': 6, 'units_2': 21, 'units_3': 11, 'units_4': 21, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0045'}
{'activation': 'tanh', 'first_units': 16, 'num_layers': 5, 'units_0': 26, 'units_1': 26, 'units_2': 1, 'units_3': 11, 'units_4': 26, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 16, 'num_layers': 5, 'units_0': 26, 'units_1': 26, 'units_2': 1, 'units_3': 11, 'units_4': 26, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0011'}


In [13]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


9/9 - 0s - 8ms/step - accuracy: 0.8407 - loss: 0.3590
Loss: 0.35904666781425476, Accuracy: 0.8407407402992249
9/9 - 0s - 9ms/step - accuracy: 0.8370 - loss: 0.3808
Loss: 0.3808307945728302, Accuracy: 0.8370370268821716
9/9 - 0s - 8ms/step - accuracy: 0.8222 - loss: 0.3704
Loss: 0.3703703284263611, Accuracy: 0.8222222328186035
